___
## <center><a id=0 style='color:red'>Импорт библиотек</a>

In [1]:
from ds_nms import funcs, data, stat_tests, utils_io, feature_select

In [2]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from typing import List, Tuple, Any, Dict
import pickle
import os
from IPython.display import clear_output
from tqdm import tqdm
import optuna
import datetime
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.base import BaseEstimator
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split, KFold, cross_validate, StratifiedGroupKFold, StratifiedKFold
from sklearn.metrics import mean_absolute_percentage_error, mean_absolute_error, mean_squared_error, r2_score, median_absolute_error
from sklearn.preprocessing import StandardScaler, RobustScaler, QuantileTransformer, Normalizer, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor, HistGradientBoostingRegressor, IsolationForest, GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.tree import DecisionTreeRegressor, ExtraTreeRegressor
from sklearn.feature_selection import RFE, SequentialFeatureSelector

np.set_printoptions(suppress=True, precision=2)
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

___
## <center><a id=1 style='color:red'>Импорт датасета</a>
 [К описанию](#99)

In [3]:
DIR = 'train_poly/'
FILE_LST = ['X_train_poly_1','target_train_1','X_train_poly_2', 'target_train_2', 'X_train_poly_3', 'target_train_3']

X_train_poly_1,target_train_1,X_train_poly_2, target_train_2, X_train_poly_3, target_train_3 = utils_io.load_data( FILE_LST, DIR )

X_train_poly_1.shape, target_train_1.shape,X_train_poly_2.shape, target_train_2.shape, X_train_poly_3.shape, target_train_3.shape

Файл data/train_poly/X_train_poly_1.pkl успешно загружен.
Файл data/train_poly/target_train_1.pkl успешно загружен.
Файл data/train_poly/X_train_poly_2.pkl успешно загружен.
Файл data/train_poly/target_train_2.pkl успешно загружен.
Файл data/train_poly/X_train_poly_3.pkl успешно загружен.
Файл data/train_poly/target_train_3.pkl успешно загружен.


((31, 20), (31,), (31, 20), (31,), (31, 20), (31,))

___
___
## <center><a id=3 style='color:red'>Выбор факторов</a>

In [ ]:
scoring=['neg_root_mean_squared_error']

feature_select.get_best_n_features(X=X_train_poly_1,
                                    y=target_train_1,
                                    selector=RFE,
                                    scoring=scoring,
                                    estimator=LinearRegression(),
                                    X_name='X_train_1',
                                    n_folds=4)

In [ ]:
feature_select.get_best_n_features(X=X_train_poly_1,
                                    y=target_train_1,
                                    selector=SequentialFeatureSelector,
                                    scoring=scoring,
                                    estimator=LinearRegression(),
                                    X_name='X_train_1',
                                    n_folds=4,
                                    direction='backward')

In [ ]:
feature_select.get_best_n_features(X=X_train_poly_1,
                                    y=target_train_1,
                                    selector=SequentialFeatureSelector,
                                    scoring=scoring,
                                    estimator=LinearRegression(),
                                    X_name='X_train_1',
                                    n_folds=4,
                                    direction='forward')

___
### <a id=5>Recursive feature elimination</a>

In [4]:
RAND_STATE = 1
scoring=['neg_root_mean_squared_error']
estimators_lst = [
    LinearRegression(),
    Ridge(alpha=0.2, random_state=RAND_STATE),
    Lasso(alpha=0.2, random_state=RAND_STATE),
    DecisionTreeRegressor(random_state=RAND_STATE),
    ExtraTreeRegressor(random_state=RAND_STATE),
    LGBMRegressor(random_state=RAND_STATE)
                ]

data_dict = {
    "X_train_rfe_1": (X_train_poly_1, target_train_1),
    "X_train_rfe_2": (X_train_poly_2, target_train_2),
    "X_train_rfe_3": (X_train_poly_3, target_train_3),
        }

___

In [5]:
result_df = feature_select.save_selected_features(data_dict=data_dict,
                                    selector=RFE,
                                    estimators=estimators_lst,
                                    scoring=scoring,
                                    output_dir=DIR)

result_df

Файл data/train_poly/f_select_2025_01_13_13_03\X_train_rfe_1_LinearRegression.pkl успешно сохранён.
Файл data/train_poly/f_select_2025_01_13_13_03\X_train_rfe_2_LinearRegression.pkl успешно сохранён.
Файл data/train_poly/f_select_2025_01_13_13_03\X_train_rfe_3_LinearRegression.pkl успешно сохранён.
Файл data/train_poly/f_select_2025_01_13_13_03\X_train_rfe_1_Ridge.pkl успешно сохранён.
Файл data/train_poly/f_select_2025_01_13_13_03\X_train_rfe_2_Ridge.pkl успешно сохранён.
Файл data/train_poly/f_select_2025_01_13_13_03\X_train_rfe_3_Ridge.pkl успешно сохранён.
Файл data/train_poly/f_select_2025_01_13_13_03\X_train_rfe_1_Lasso.pkl успешно сохранён.
Файл data/train_poly/f_select_2025_01_13_13_03\X_train_rfe_2_Lasso.pkl успешно сохранён.
Файл data/train_poly/f_select_2025_01_13_13_03\X_train_rfe_3_Lasso.pkl успешно сохранён.
Файл data/train_poly/f_select_2025_01_13_13_03\X_train_rfe_1_DecisionTreeRegressor.pkl успешно сохранён.
Файл data/train_poly/f_select_2025_01_13_13_03\X_train_rfe_2_

,number,value,datetime_start,datetime_complete,duration,params_n_features_to_select,system_attrs_grid_id,system_attrs_search_space,state,data_name,estimator
0,0,17673.956244,2025-01-13 13:03:02.092679,2025-01-13 13:03:02.112670,0 days 00:00:00.019991,7,0,"{'n_features_to_select': [1, 2, 3, 4, 5, 6, 7,...",COMPLETE,X_train_rfe_1,LinearRegression
1,1,20739.151828,2025-01-13 13:03:02.113671,2025-01-13 13:03:02.134672,0 days 00:00:00.021001,9,1,"{'n_features_to_select': [1, 2, 3, 4, 5, 6, 7,...",COMPLETE,X_train_rfe_1,LinearRegression
2,2,139661.823536,2025-01-13 13:03:02.135670,2025-01-13 13:03:02.160671,0 days 00:00:00.025001,18,2,"{'n_features_to_select': [1, 2, 3, 4, 5, 6, 7,...",COMPLETE,X_train_rfe_1,LinearRegression
3,3,43013.535711,2025-01-13 13:03:02.162671,2025-01-13 13:03:02.186671,0 days 00:00:00.024000,15,3,"{'n_features_to_select': [1, 2, 3, 4, 5, 6, 7,...",COMPLETE,X_train_rfe_1,LinearRegression
4,4,18150.137330,2025-01-13 13:03:02.187670,2025-01-13 13:03:02.208670,0 days 00:00:00.021000,4,4,"{'n_features_to_select': [1, 2, 3, 4, 5, 6, 7,...",COMPLETE,X_train_rfe_1,LinearRegression
...,...,...,...,...,...,...,...,...,...,...,...
337,14,923.678798,2025-01-13 13:03:14.700481,2025-01-13 13:03:14.777474,0 days 00:00:00.076993,11,14,"{'n_features_to_select': [1, 2, 3, 4, 5, 6, 7,...",COMPLETE,X_train_rfe_3,LGBMRegressor
338,15,923.678798,2025-01-13 13:03:14.778481,2025-01-13 13:03:14.833476,0 days 00:00:00.054995,19,15,"{'n_features_to_select': [1, 2, 3, 4, 5, 6, 7,...",COMPLETE,X_train_rfe_3,LGBMRegressor
339,16,923.678798,2025-01-13 13:03:14.834478,2025-01-13 13:03:14.979477,0 days 00:00:00.144999,3,16,"{'n_features_to_select': [1, 2, 3, 4, 5, 6, 7,...",COMPLETE,X_train_rfe_3,LGBMRegressor
340,17,923.678798,2025-01-13 13:03:14.980477,2025-01-13 13:03:15.063478,0 days 00:00:00.083001,18,17,"{'n_features_to_select': [1, 2, 3, 4, 5, 6, 7,...",COMPLETE,X_train_rfe_3,LGBMRegressor


In [31]:
min_result_df_1 = result_df[result_df['data_name'] == 'X_train_rfe_1' ]
min_result_df_1 = min_result_df_1[min_result_df_1['value'] == min_result_df_1['value'].min()]
min_result_df_1

,number,value,datetime_start,datetime_complete,duration,params_n_features_to_select,system_attrs_grid_id,system_attrs_search_space,state,data_name,estimator
119,5,11115.340033,2025-01-13 12:12:58.361451,2025-01-13 12:12:58.392457,0 days 00:00:00.031006,8,5,"{'n_features_to_select': [1, 2, 3, 4, 5, 6, 7,...",COMPLETE,X_train_rfe_1,Lasso


In [32]:
min_result_df_2 = result_df[result_df['data_name'] == 'X_train_rfe_2' ]
min_result_df_2 = min_result_df_2[min_result_df_2['value'] == min_result_df_2['value'].min()]
min_result_df_2

,number,value,datetime_start,datetime_complete,duration,params_n_features_to_select,system_attrs_grid_id,system_attrs_search_space,state,data_name,estimator
139,6,1237.755061,2025-01-13 12:12:59.004451,2025-01-13 12:12:59.035460,0 days 00:00:00.031009,8,6,"{'n_features_to_select': [1, 2, 3, 4, 5, 6, 7,...",COMPLETE,X_train_rfe_2,Lasso


In [33]:
min_result_df_3 = result_df[result_df['data_name'] == 'X_train_rfe_3' ]
min_result_df_3 = min_result_df_3[min_result_df_3['value'] == min_result_df_3['value'].min()]
min_result_df_3


,number,value,datetime_start,datetime_complete,duration,params_n_features_to_select,system_attrs_grid_id,system_attrs_search_space,state,data_name,estimator
99,4,527.330161,2025-01-13 12:12:57.911458,2025-01-13 12:12:57.926451,0 days 00:00:00.014993,9,4,"{'n_features_to_select': [1, 2, 3, 4, 5, 6, 7,...",COMPLETE,X_train_rfe_3,Ridge


In [34]:
DIR_LOAD = 'train_poly/2025_01_13_12_12/'
FILE_LST = ['X_train_rfe_1_Lasso', 'X_train_rfe_2_Lasso','X_train_rfe_3_Ridge']

X_train_rfe_1_Lasso, X_train_rfe_2_Lasso, X_train_rfe_3_Ridge = utils_io.load_data( FILE_LST, DIR_LOAD )

X_train_rfe_1_Lasso.shape, X_train_rfe_2_Lasso.shape, X_train_rfe_3_Ridge.shape

Файл data/train_poly/2025_01_13_12_12/X_train_rfe_1_Lasso.pkl успешно загружен.
Файл data/train_poly/2025_01_13_12_12/X_train_rfe_2_Lasso.pkl успешно загружен.
Файл data/train_poly/2025_01_13_12_12/X_train_rfe_3_Ridge.pkl успешно загружен.


((31, 8), (31, 8), (31, 9))

___
### <a id=5>SequentialFeatureSelector</a>

In [ ]:
RAND_STATE = 1
scoring=['neg_root_mean_squared_error']
estimators_lst = [
    LinearRegression(),
    Ridge(alpha=0.2, random_state=RAND_STATE),
    Lasso(alpha=0.2, random_state=RAND_STATE),
    DecisionTreeRegressor(random_state=RAND_STATE),
    ExtraTreeRegressor(random_state=RAND_STATE),
    LGBMRegressor(random_state=RAND_STATE)
                ]

data_dict = {
    "X_train_rfe_1": (X_train_poly_1, target_train_1),
    "X_train_rfe_2": (X_train_poly_2, target_train_2),
    "X_train_rfe_3": (X_train_poly_3, target_train_3),
        }

___

In [ ]:
result_df = feature_select.save_selected_features(data_dict=data_dict,
                                    selector=SequentialFeatureSelector,
                                    estimators=estimators_lst,
                                    scoring=scoring,
                                    output_dir=DIR,
                                    direction='backward')

result_df

In [36]:
min_result_df_1 = result_df[result_df['data_name'] == 'X_train_rfe_1' ]
min_result_df_1 = min_result_df_1[min_result_df_1['value'] == min_result_df_1['value'].min()]
min_result_df_1

,number,value,datetime_start,datetime_complete,duration,params_n_features_to_select,system_attrs_grid_id,system_attrs_search_space,state,data_name,estimator
73,16,10612.183263,2025-01-13 12:36:12.792489,2025-01-13 12:36:13.945488,0 days 00:00:01.152999,7,16,"{'n_features_to_select': [1, 2, 3, 4, 5, 6, 7,...",COMPLETE,X_train_rfe_1,Ridge


In [37]:
min_result_df_2 = result_df[result_df['data_name'] == 'X_train_rfe_2' ]
min_result_df_2 = min_result_df_2[min_result_df_2['value'] == min_result_df_2['value'].min()]
min_result_df_2

,number,value,datetime_start,datetime_complete,duration,params_n_features_to_select,system_attrs_grid_id,system_attrs_search_space,state,data_name,estimator
27,8,1846.663942,2025-01-13 12:35:14.471493,2025-01-13 12:35:14.809494,0 days 00:00:00.338001,2,8,"{'n_features_to_select': [1, 2, 3, 4, 5, 6, 7,...",COMPLETE,X_train_rfe_2,LinearRegression


In [38]:
min_result_df_3 = result_df[result_df['data_name'] == 'X_train_rfe_3' ]
min_result_df_3 = min_result_df_3[min_result_df_3['value'] == min_result_df_3['value'].min()]
min_result_df_3


,number,value,datetime_start,datetime_complete,duration,params_n_features_to_select,system_attrs_grid_id,system_attrs_search_space,state,data_name,estimator
222,13,527.207107,2025-01-13 12:40:29.358802,2025-01-13 12:40:29.781802,0 days 00:00:00.423000,2,13,"{'n_features_to_select': [1, 2, 3, 4, 5, 6, 7,...",COMPLETE,X_train_rfe_3,DecisionTreeRegressor


In [40]:
DIR_LOAD = 'train_poly/2025_01_13_12_12/2025_01_13_12_34/'
FILE_LST = ['X_train_rfe_1_Ridge', 'X_train_rfe_2_LinearRegression','X_train_rfe_3_DecisionTreeRegressor']

X_train_rfe_1_Ridge, X_train_rfe_2_LinearRegression, X_train_rfe_3_DecisionTreeRegressor = utils_io.load_data( FILE_LST, DIR_LOAD )

X_train_rfe_1_Ridge.shape, X_train_rfe_2_LinearRegression.shape, X_train_rfe_3_DecisionTreeRegressor.shape

Файл data/train_poly/2025_01_13_12_12/2025_01_13_12_34/X_train_rfe_1_Ridge.pkl успешно загружен.
Файл data/train_poly/2025_01_13_12_12/2025_01_13_12_34/X_train_rfe_2_LinearRegression.pkl успешно загружен.
Файл data/train_poly/2025_01_13_12_12/2025_01_13_12_34/X_train_rfe_3_DecisionTreeRegressor.pkl успешно загружен.


((31, 7), (31, 2), (31, 2))